In [53]:
import torch as t
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms

# 1. Basic autograd example 1

In [18]:
x = t.tensor([1.], requires_grad=True)
w = t.tensor([3.], requires_grad=True)
b = t.tensor([1.], requires_grad=True)

#x = t.tensor([1.], requires_grad=False)
#w = t.tensor([3.], requires_grad=False)
#b = t.tensor([1.], requires_grad=False)

In [19]:
y = w*x+b

In [20]:
#Compute gradients
y.backward()

In [21]:
# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad) # b.grad = 1 

tensor([3.])
tensor([1.])
tensor([1.])


# 2. Basic autograd example 2 - Linear regression

In [25]:
x  = t.randn(10, 3)
y = t.randn(10, 2)

In [28]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print('w:', linear.weight)
print('b:', linear.bias)

w: Parameter containing:
tensor([[-0.3620,  0.1297, -0.0704],
        [ 0.5312, -0.4227,  0.3983]], requires_grad=True)
b: Parameter containing:
tensor([-0.0200,  0.2894], requires_grad=True)


In [29]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = t.optim.SGD(linear.parameters(), lr = 0.001)

In [31]:
#forward pass
pred = linear(x)
pred

tensor([[-0.2413,  0.3939],
        [-0.6658,  2.0078],
        [-0.2918,  0.8595],
        [-0.4703,  0.9691],
        [-0.4649,  1.1805],
        [ 0.0680, -0.0405],
        [-0.3033,  0.5804],
        [ 0.0091,  0.8894],
        [-0.0304,  0.0084],
        [-0.7110,  1.1901]], grad_fn=<AddmmBackward>)

In [36]:
# compute loss
loss = criterion(pred, y)
print('loss:', loss.item())

loss: 1.7566441297531128


In [37]:
# backward pass
loss.backward()

In [38]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[-0.9617, -0.2657, -0.3693],
        [ 0.8084, -0.5895,  0.0793]])
dL/db:  tensor([-0.4315,  0.6915])


In [40]:
# 1-step gradient descent.
optimizer.step()

In [41]:
pred = linear(x)
loss = criterion(pred, y)
print("loss:", loss.item())

loss: 1.753842830657959


# 3. Loading data from numpy

In [45]:
# Create a numpy array.
x = np.array([[1,2], [3,4]])

In [49]:
# Convert the numpy array to a torch tensor.
y = t.from_numpy(x)
y

tensor([[1, 2],
        [3, 4]])

In [50]:
# Convert the torch tensor to a numpy array.
z = y.numpy()
z

array([[1, 2],
       [3, 4]])

#  4. Input pipline 

In [54]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root= '../../data/',
                                            train = True,
                                             transform = transforms.ToTensor(),
                                            download = True)

In [55]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]

In [57]:
label

6

In [58]:
train_loader = t.utils.data.DataLoader(dataset=train_dataset,
                                      shuffle = True,
                                      batch_size = 64)

In [60]:
for image, labels in train_loader:
    # Training code to be written here
    break

# 5. Input pipline for custom dataset     

In [61]:
# You should build your custom dataset as below.
class CustomDataset(t.utils.data.Dataset):
    def __init__(self):
        #TODO
        # 1. Initilize paths and filenames of the custom data
        pass
    def __getitem__(self, index):
        #TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        total_size_of_your_dataset = 0
        return total_size_of_your_dataset

In [62]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = t.utils.data.DataLoader(dataset=custom_dataset,
                                      batch_size = 64,
                                      shuffle = True)

ValueError: num_samples should be a positive integeral value, but got num_samples=0

# 6. Pretrained model

In [63]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet101(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /home/karthik/.torch/models/resnet101-5d3b4d8f.pth
178728960.0 bytes


In [64]:
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 10) # 10 is the embedding size

In [68]:
# Forward prop
images = t.randn(64, 3, 224, 224)
output = resnet(images)
print (output.size())

torch.Size([64, 10])


#  7. Save and load the model 

In [69]:
# Save and load the entire model.
t.save(resnet, "model.ckpt")
resnet = t.load("model.ckpt")

In [72]:
# Save and load only the model parameters (recommended).
t.save(resnet.state_dict(), "params.ckpt")
resnet.load_state_dict(t.load("params.ckpt"))